In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import func
import torchopt
from torch.utils.data import DataLoader
import functorch

lr = 1e-3


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


class Loader(DataLoader):
    def __init__(self):
        super().__init__(range(100), batch_size=10)

    def __iter__(self):
        for i in range(10):
            yield torch.randn(10).float(), torch.randint(0, 10, (10,)).float()


net = Net()  # init
loader = Loader()
optimizer = torchopt.adam(lr)

# params = dict(net.named_parameters())  # get network parameters
model, params = functorch.make_functional(net)
opt_state = optimizer.init(params)  # init optimizer

for data in loader:
    xs, ys = data  # get data
# xs, ys = next(loader)  # get data
pred = model(params,xs)  # forward
loss = F.cross_entropy(pred, ys)  # compute loss

grads = torch.autograd.grad(loss, params)  # compute gradients
updates, opt_state = optimizer.update(grads, opt_state)  # get updates
params = torchopt.apply_updates(params, updates)  # update network parameters

In [8]:
net = Net()  # init
loader = Loader()
optimizer = torchopt.FuncOptimizer(torchopt.adam())      # wrap with `torchopt.FuncOptimizer`

model, params = functorch.make_functional(net)           # use functorch extract network parameters

for xs, ys in loader:                                    # get data
    pred = model(params, xs)                             # forward
    loss = F.cross_entropy(pred, ys)                     # compute loss

    params = optimizer.step(loss, params)                # update network parameters

/home/aris/mambaforge/envs/grab/lib/python3.10/site-packages/torch/_functorch/deprecated.py:100: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.make_functional is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.functional_call instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('make_functional', 'torch.func.functional_call')


In [ ]:
torchopt.schedule